<a href="https://colab.research.google.com/github/Nmg1994/Beehive_habitat_suitability/blob/main/Beehive_habitat_suitability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing necessary modules

In [ ]:
!pip install scikit-survival pdpbox matplotlib
!pip install tqdm
!pip install randomSurvivalForest
!pip install rasterio

# Importing necessary libraries

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import (
    concordance_index_censored,
    integrated_brier_score
)

from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.util import Surv
from sklearn.inspection import permutation_importance
import rasterio
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.isotonic import IsotonicRegression
from scipy.interpolate import interp1d

# Mounting Google Drive

In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Preprocessing air quality data for processing them in ArcGIS Pro

In [ ]:
Stations = pd.read_csv("/content/drive/My Drive/Air quality data/liste-des-stations-rsqa.csv")

for y in range(2017, 2022):
  air_quality = pd.read_csv(f"/content/drive/My Drive/Air quality data/rsqa-multi-polluants{y}.csv", low_memory=False)
  stations = air_quality['NO_POSTE'].unique()

  All_stations = pd.DataFrame(columns=['Station', 'latitude', 'longitude','PM2,5_'+str(y), 'O3_'+str(y)])

  i = 1
  for station in stations:
    each_station_AirQuality = air_quality[air_quality['NO_POSTE'] == station][['PM2,5','O3']].apply(pd.to_numeric, errors='coerce')
    All_stations.loc[i,'Station'] = station
    All_stations.loc[i,'latitude'] = Stations[Stations['numero_station'] == station]['latitude'].iloc[0]
    All_stations.loc[i,'longitude'] = Stations[Stations['numero_station'] == station]['longitude'].iloc[0]
    All_stations.loc[i, 'PM2,5_'+str(y)] = each_station_AirQuality.mean(axis = 0)['PM2,5']
    All_stations.loc[i, 'O3_'+str(y)] = each_station_AirQuality.mean(axis = 0)['O3']
    i +=1

  All_stations.to_csv(f"/content/drive/My Drive/Air quality data/Air_Quality_{y}.csv", index=False)

# Preprocessing data

In [ ]:
variables = ['NDVI','O3', 'PM 2,5','Road density','Road proximity']
variables_df = pd.DataFrame()
for var in variables:

  if var == 'Road density' or var == 'Road proximity':
    df = pd.read_excel(f"/content/drive/My Drive/COX_RSF_Paper/{var}.xls")
    preprocessed_df = df[['ColonyID','MEAN']].rename(columns={'MEAN': f'{var}'})
    variables_df = pd.concat([variables_df, preprocessed_df], axis = 1)
  else:
    for y in range(2017, 2022):
      df = pd.read_excel(f"/content/drive/My Drive/COX_RSF_Paper/{var}_{y}.xls")
      preprocessed_df = df[['ColonyID','MEAN']].rename(columns={'MEAN': f'{var}_{y}'})
      variables_df = pd.concat([variables_df, preprocessed_df], axis = 1)

variables_df = variables_df.loc[:, ~variables_df.columns.duplicated()]
variables_df.to_csv('/content/drive/My Drive/COX_RSF_Paper/variables_excep_LandC.csv', index=True)

In [ ]:
LandCover_all_years= pd.DataFrame()

for y in range(2017, 2022):
  df = pd.read_excel(f"/content/drive/My Drive/COX_RSF_Paper/LandCover_{y}.xls")
  df = df.drop(columns= ['OBJECTID']).reset_index(drop=True)

  df['Sum'] = df[['WINTER_WHEAT', 'SPRING_WHEAT', 'SOYBEANS',
        'CORN', 'PASTURE_FORAGES', 'GREENHOUSES', 'WETLAND', 'WATER',
        'SHRUBLAND', 'CONIFEROUS', 'BROADLEAF', 'MIXEDWOOD', 'URBAN_DEVELOPED',
        'EXPOSED_LAND_BARREN']].sum(axis = 1)

  columns_to_divide = ['WINTER_WHEAT', 'SPRING_WHEAT', 'SOYBEANS',
        'CORN', 'PASTURE_FORAGES', 'GREENHOUSES', 'WETLAND', 'WATER',
        'SHRUBLAND', 'CONIFEROUS', 'BROADLEAF', 'MIXEDWOOD', 'URBAN_DEVELOPED',
        'EXPOSED_LAND_BARREN']

  df[columns_to_divide] = df[columns_to_divide].div(df['Sum'], axis=0)

  Preprocessed_landC_df = df[['COLONYID','WATER','URBAN_DEVELOPED']].rename(columns={'COLONYID': 'ColonyID', 'WATER': f'Percentage of water_{y}', 'URBAN_DEVELOPED': f'Percentage of urban areas_{y}'})
  LandCover_all_years = pd.concat([LandCover_all_years, Preprocessed_landC_df], axis = 1)

LandCover_all_years = LandCover_all_years.loc[:, ~LandCover_all_years.columns.duplicated()]
LandCover_all_years.to_csv('/content/drive/My Drive/COX_RSF_Paper/LandCover_all_years.csv', index=True)

All_variables = pd.merge(variables_df, LandCover_all_years, on='ColonyID', how='inner')
All_variables.to_csv('/content/drive/My Drive/COX_RSF_Paper/All_variables.csv', index=True)

In [ ]:
Montreal_dataset = pd.read_excel("/content/drive/My Drive/COX_RSF_Paper/Beehive_dataset.xlsx")
Montreal_dataset_2017_2021 = Montreal_dataset[(Montreal_dataset['Year'] != 2022)]
Montreal_dataset_2017_2021_filtered = Montreal_dataset_2017_2021.loc[Montreal_dataset_2017_2021.groupby('ColonyID')['Year'].idxmax()]
Montreal_dataset_cleaned = Montreal_dataset_2017_2021_filtered[['Join_Count','ColonyID','Year','Status','Place']].reset_index(drop=True)

Montreal_dataset_cleaned.loc[Montreal_dataset_cleaned['Status'] == 'Dead', 'event']=1
Montreal_dataset_cleaned.loc[Montreal_dataset_cleaned['Status'] == 'Healthy', 'event']=0
Montreal_dataset_cleaned['event'] = Montreal_dataset_cleaned['event'].astype(int)

Montreal_dataset_cleaned.loc[Montreal_dataset_cleaned['Place'] == 'Roof', 'Place']=1
Montreal_dataset_cleaned.loc[Montreal_dataset_cleaned['Place'] == 'Ground', 'Place']=0

Montreal_dataset_cleaned['Year'] = Montreal_dataset_cleaned['Year'] - 2016 # assuming that the year 2017 is the first year of beehive creation; in other words, the age of the beehive is considered since the year 2017
Montreal_dataset_cleaned['Year'] = Montreal_dataset_cleaned['Year'].astype(int)
Montreal_dataset_cleaned = Montreal_dataset_cleaned.rename(columns={'Join_Count': 'Surrounding hive count', 'Year':'time', 'Place':'Hive placement'})
Montreal_Beehive_df = Montreal_dataset_cleaned.drop(columns= ['Status']).reset_index(drop=True)
Montreal_Beehive_df.to_csv('/content/drive/My Drive/COX_RSF_Paper/Preprocessed_Beehive_dataset.csv', index=False)
pd.merge(Montreal_Beehive_df, All_variables, on='ColonyID', how='inner').to_csv('/content/drive/My Drive/COX_RSF_Paper/Preprocessed_Beehive_dataset_with_variables.csv', index=False)

In [ ]:
Preprocessed_Beehive_dataset_with_variables = pd.read_csv("/content/drive/My Drive/COX_RSF_Paper/Preprocessed_Beehive_dataset_with_variables.csv")
Base_df = Preprocessed_Beehive_dataset_with_variables[['Surrounding hive count','ColonyID','time','Hive placement','event','Road density','Road proximity']].copy()

Final_preprocessed_df = pd.DataFrame()

# Loop through the years and process data
for y in range(2017, 2022):
    Base_df['Year'] = y
    # Find columns containing the year
    annual_columns = Preprocessed_Beehive_dataset_with_variables.filter(regex=f'_{y}').columns.tolist()
    annual_df = Preprocessed_Beehive_dataset_with_variables[annual_columns].copy()
    annual_df.columns = annual_df.columns.str.replace(f'_{y}', '', regex=True)

    temp_df = pd.concat([Base_df.reset_index(drop=True), annual_df.reset_index(drop=True)], axis=1)
    Final_preprocessed_df = pd.concat([Final_preprocessed_df, temp_df], axis=0, ignore_index=True)

# Save the final preprocessed DataFrame to a CSV file
Final_preprocessed_df.to_csv('/content/drive/My Drive/COX_RSF_Paper/Final_preprocessed_df.csv', index=False)

# Spearman correlation and multi-collinearity analyses

In [ ]:
Features = Final_preprocessed_df.drop(columns= ['ColonyID','event', 'time'])
# Calculating the Spearman correlation matrix
corr_matrix = Features.corr(method='spearman', numeric_only=True)

# Masking the upper triangle excluding the diagonal
mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)

In [ ]:
# Setting up the matplotlib figure
plt.figure(figsize=(10, 8))

# Plotting the heatmap with the mask and correct aspect ratio for the variables
sns.heatmap(corr_matrix,  annot = True, mask=mask, cmap='coolwarm', fmt='.2f', square=True, cbar_kws={"shrink": 0.75}, linewidths=0.1)

plt.title('Spearman correlation analysis between the features')
plt.show()

In [ ]:
# Calculating VIF and Tolerance
def calculate_vif(df):
  # Adding a constant column for the intercept
  df_with_constant = sm.add_constant(df)

  # Calculating VIF for each feature
  vif_data = pd.DataFrame()
  vif_data['Feature'] = df.columns
  vif_data['VIF'] = [variance_inflation_factor(df_with_constant.values, i+1) for i in range(len(df.columns))]
  vif_data['Tolerance'] = 1 / vif_data['VIF']

  return vif_data

# Using the function to calculate VIF and Tolerance
VIF = calculate_vif(Features)
# Converting to integer format
VIF["VIF"] = VIF["VIF"].apply(lambda x: int(round(x)))
print('VIF evaluation')
print(VIF)

features_high_vif = VIF[VIF['VIF'] > 5]
columns_to_drop = np.array(features_high_vif['Feature'])


# Splitting data into training and testing datasets

In [ ]:
#Splitting datasets after dropping the features with the VIF value more than 10
def splitting_data(dataframe, columns_to_drop):
  columns_to_drop_all = np.concatenate((columns_to_drop , np.array(['ColonyID', 'event', 'time'])), axis = 0)
  Considered_features = [col for col in dataframe.columns if col not in columns_to_drop_all]
  print('Considered_features: ', Considered_features)

  X = np.array(dataframe.drop(columns= columns_to_drop_all))

  # Structured array of target variable
  Y = Surv.from_arrays(dataframe['event'], dataframe['time'])  # survival outcome (event and time)

  # Initial split: Training and Testing datasets (70:30)
  X_training, X_testing, Y_training, Y_testing = train_test_split(X, Y, test_size=0.3, shuffle=True, random_state=0)

  # Shape of datasets
  print('X_training shape: ', X_training.shape)
  print('Y_training shape: ', Y_training.shape)
  print('X_testing shape: ', X_testing.shape)
  print('Y_testing shape: ', Y_testing.shape)

  return X_training, X_testing, Y_training, Y_testing, Considered_features, X, Y

In [ ]:
# Applying splitting_data function to split training dataset from testing
X_training, X_testing, Y_training, Y_testing, Considered_features, X_total, Y_total= splitting_data(Final_preprocessed_df, columns_to_drop)

# Training RSF and Cox models and evaluating their accuracy

RSF Model

In [ ]:
# Optimizing hyper-parameters for the RSF using California dataset
parameters = {'n_estimators':np.arange(100,500,50), 'max_depth':np.arange(1,30,4)}
RSF = RandomSurvivalForest(random_state=0, bootstrap = True)
RSF_model_gs = GridSearchCV(RSF, parameters, cv=5, return_train_score=True, n_jobs=-1)
RSF_model_gs.fit(X_training, Y_training)
RSF_model_gs.best_params_

# Printing the best hyperparameters found
print("Best hyperparameters:", RSF_model_gs.best_params_)
print("\n")

# Printing the best cross-validated score
print("Best cross-validated score (concordance index):", RSF_model_gs.best_score_)

/usr/local/lib/python3.11/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best hyperparameters: {'max_depth': 9, 'n_estimators': 250}


Best cross-validated score (concordance index): 0.6067007325371647


In [ ]:
# Fitting a RSF model to the training dataset having found the optimized values for the hyper-parameters
# the configuration including beehive management criteria
RSF_model = RandomSurvivalForest(random_state=0, n_estimators = 250, max_depth = 9, bootstrap = True)
RSF_model.fit(X_training, Y_training)

RandomSurvivalForest(max_depth=9, n_estimators=250, random_state=0)

Cox model

In [ ]:
# Fitting a Cox Proportional Hazards model to the training dataset
cox_model = CoxPHSurvivalAnalysis()
cox_model.fit(X_training, Y_training)

CoxPHSurvivalAnalysis()

Evaluation of models' accuracy

In [ ]:
# RSF model
def calibrate_rsf_model(rsf_model, X_training, Y_training):
  # Predicting survival probabilities on the training data
  pred_surv_probs = rsf_model.predict_survival_function(X_training, return_array=True)

  # Preparing the true survival probabilities for calibration
  Y_training_real = Surv.from_arrays(event=Y_training['event'], time=Y_training['time'])
  unique_times_real = np.unique(Y_training['time'])
  mean_true_surv_probs = np.array([np.mean(Y_training['time'] >= t) for t in unique_times_real])

  # Interpolating predicted probabilities to align with the true survival times
  interp_func = interp1d(rsf_model.unique_times_, np.mean(pred_surv_probs, axis=0), kind='nearest', fill_value='extrapolate')
  pred_surv_probs_interp = interp_func(unique_times_real)

  # Applying Isotonic regression for calibration
  iso_reg = IsotonicRegression(out_of_bounds='clip')
  iso_reg.fit(pred_surv_probs_interp, mean_true_surv_probs)
  calibrated_pred_surv_probs = iso_reg.transform(pred_surv_probs_interp)

  return calibrated_pred_surv_probs, unique_times_real

def evaluation_rsf_model(model, X_train, X_test, Y_train, Y_test):
  # Calibrating the model using the training set
  calibrated_pred_surv_probs, unique_times_real = calibrate_rsf_model(model, X_train, Y_train)

  # Calculating the Concordance index (C-index)
  c_index = concordance_index_censored(Y_test["event"], Y_test["time"], model.predict(X_test))
  print("Concordance index (C-index):", c_index[0])

  # Getting the probability of event-free survival for the testing dataset
  surv_funcs = model.predict_survival_function(X_test)


  # Finding the range of follow-up times in Y_test
  min_time_test = int(Y_test['time'].min())
  max_time_test = int(Y_test['time'].max())

  # Ensuring times align with follow-up range
  times = np.array([t for t in model.unique_times_ if min_time_test <= t < max_time_test])
  preds = np.array([[fn(t) for t in times] for fn in surv_funcs])

  # Computing the integrated Brier score (IBS) using calibrated probabilities
  ibs = integrated_brier_score(Y_train, Y_test, preds, times)
  print("Integrated Brier Score (IBS):", ibs)


# Cox model
def calibrate_cox_model(cox_model, X_training, Y_training):

    # Predicting survival functions for training data
    surv_funcs = cox_model.predict_survival_function(X_training)
    unique_times_real = np.unique(Y_training["time"])

    # Calculating mean predicted survival probabilities at unique times
    pred_surv_probs = np.array([[fn(t) for t in unique_times_real] for fn in surv_funcs])
    mean_pred_surv_probs = pred_surv_probs.mean(axis=0)

    # Preparing the true survival probabilities for calibration
    mean_true_surv_probs = np.array([np.mean(Y_training["time"] >= t) for t in unique_times_real])

    # Applying Isotonic regression for calibration
    iso_reg = IsotonicRegression(out_of_bounds="clip")
    iso_reg.fit(mean_pred_surv_probs, mean_true_surv_probs)
    calibrated_pred_surv_probs = iso_reg.transform(mean_pred_surv_probs)

    return calibrated_pred_surv_probs, unique_times_real


def evaluation_cox_model(cox_model, X_train, X_test, Y_train, Y_test):
    # Calibrating the Cox model
    calibrated_pred_surv_probs, unique_times_real = calibrate_cox_model(cox_model, X_train, Y_train)

    # Concordance Index (C-index)
    c_index = concordance_index_censored(Y_test["event"], Y_test["time"], cox_model.predict(X_test))
    print("Concordance Index (C-index):", c_index[0])

    # Predicting survival functions for test data
    surv_funcs = cox_model.predict_survival_function(X_test)

    # Range of follow-up times in Y_test
    min_time_test = int(Y_test["time"].min())
    max_time_test = int(Y_test["time"].max())

    # Ensuring times align with follow-up range
    times = np.array([t for t in cox_model.unique_times_ if min_time_test <= t < max_time_test])
    preds = np.array([[fn(t) for t in times] for fn in surv_funcs])

    # Computing the integrated Brier score (IBS) using calibrated probabilities
    ibs = integrated_brier_score(Y_train, Y_test, preds, times)
    print("Integrated Brier Score (IBS):", ibs)

Model evaluation results on both training and test datasets

In [ ]:
# Accuracy on the training and test dataset
# Using the RSF model
print("Accuracy of the RSF model on the training dataset")
evaluation_rsf_model(model = RSF_model, X_train = X_training, X_test = X_training, Y_train = Y_training, Y_test = Y_training)
print("\nAccuracy of the RSF model on the test dataset")
evaluation_rsf_model(model = RSF_model, X_train = X_training, X_test = X_testing, Y_train = Y_training, Y_test = Y_testing)

# Using the Cox model
print("\nAccuracy of the Cox model on the training dataset")
evaluation_cox_model(cox_model = cox_model, X_train = X_training, X_test = X_training, Y_train = Y_training, Y_test = Y_training)
print("\nAccuracy of the Cox model on the test dataset")
evaluation_cox_model(cox_model = cox_model, X_train = X_training, X_test = X_testing, Y_train = Y_training, Y_test = Y_testing)


# Variable importance assessment

In [ ]:
def feature_importance_evaluation_RSF_model(RSF_model, X, Y, dataframe):
  # Evaluating feature importance
  result = permutation_importance(RSF_model, X, Y, n_repeats=15)
  feature_importance_df = pd.DataFrame(
      {
          k: result[k]
          for k in ("importances_mean", "importances_std",)
      },
      index=dataframe.columns,
  ).sort_values(by="importances_mean", ascending=False)

  # Plotting error bars and line graph
  plt.figure(figsize=(12, 5))
  plt.gca().set_facecolor('white')
  plt.grid(False)

  # Error bars
  plt.errorbar(
      x=feature_importance_df.index,
      y=feature_importance_df['importances_mean'],
      yerr=feature_importance_df['importances_std'],
      fmt='o', capsize=5, color='b'
  )

  # Line graph to connect the points
  plt.plot(
      feature_importance_df.index,
      feature_importance_df['importances_mean'],
      linestyle='-', marker=''
  )


  plt.xlabel('Features', fontsize = 12)
  plt.ylabel('Mean importance', fontsize = 12)
  plt.title('Importance of features', fontsize = 14)
  plt.xticks(rotation=45, ha='right', fontsize = 12)
  plt.yticks(fontsize = 12)
  plt.tight_layout()
  plt.show()

  return feature_importance_df

In [ ]:
# Applying the feature_importance_evaluation_RSF_model function
feature_importance_df = feature_importance_evaluation_RSF_model(RSF_model = RSF_model, X = X_total, Y = Y_total, dataframe = Final_preprocessed_df[Considered_features])

# Calculating the survival probability of each pixel in the study area and providing beehive habitat suitability maps

In [ ]:
# Defining raster file paths
raster_paths = {
    'Road_proximity': '/content/drive/My Drive/COX_RSF_Paper/Road proximity.tif',
    'Road_density': '/content/drive/My Drive/COX_RSF_Paper/Road density.tif',
    'NDVI_2017': '/content/drive/My Drive/COX_RSF_Paper/NDVI 2017.tif',
    'NDVI_2018': '/content/drive/My Drive/COX_RSF_Paper/NDVI 2018.tif',
    'NDVI_2019': '/content/drive/My Drive/COX_RSF_Paper/NDVI 2019.tif',
    'NDVI_2020': '/content/drive/My Drive/COX_RSF_Paper/NDVI 2020.tif',
    'NDVI_2021': '/content/drive/My Drive/COX_RSF_Paper/NDVI 2021.tif',
    'Surrounding_2017': '/content/drive/My Drive/COX_RSF_Paper/Surrounding beehives 2017.tif',
    'Surrounding_2018': '/content/drive/My Drive/COX_RSF_Paper/Surrounding beehives 2018.tif',
    'Surrounding_2019': '/content/drive/My Drive/COX_RSF_Paper/Surrounding beehives 2019.tif',
    'Surrounding_2020': '/content/drive/My Drive/COX_RSF_Paper/Surrounding beehives 2020.tif',
    'Surrounding_2021': '/content/drive/My Drive/COX_RSF_Paper/Surrounding beehives 2021.tif'
}

# Function to read raster data
def read_raster(file_path):
    with rasterio.open(file_path) as src:
        return src.read(1), src.transform

# Reading raster datasets
Road_proximity_data, transform_Road_proximity = read_raster(raster_paths['Road_proximity'])
Road_density_data, transform_Road_density = read_raster(raster_paths['Road_density'])
NDVI_2017_data, transform_NDVI_2017 = read_raster(raster_paths['NDVI_2017'])
NDVI_2018_data, transform_NDVI_2018 = read_raster(raster_paths['NDVI_2018'])
NDVI_2019_data, transform_NDVI_2019 = read_raster(raster_paths['NDVI_2019'])
NDVI_2020_data, transform_NDVI_2020 = read_raster(raster_paths['NDVI_2020'])
NDVI_2021_data, transform_NDVI_2021 = read_raster(raster_paths['NDVI_2021'])
Surrounding_2017_data, transform_Surrounding_2017 = read_raster(raster_paths['Surrounding_2017'])
Surrounding_2018_data, transform_Surrounding_2018 = read_raster(raster_paths['Surrounding_2018'])
Surrounding_2019_data, transform_Surrounding_2019 = read_raster(raster_paths['Surrounding_2019'])
Surrounding_2020_data, transform_Surrounding_2020 = read_raster(raster_paths['Surrounding_2020'])
Surrounding_2021_data, transform_Surrounding_2021 = read_raster(raster_paths['Surrounding_2021'])

rows = NDVI_2017_data.shape[0]
cols = NDVI_2017_data.shape[1]

with rasterio.open(raster_paths['NDVI_2021']) as src:
   data_crs = src.crs

# Computing survival probabilities for each year
survival_probs = np.zeros((rows, cols)).reshape(-1,1)

Road_proximity_data_flat = Road_proximity_data.reshape(-1,1)
Road_density_data_flat = Road_density_data.reshape(-1,1)
NDVI_2017_flat = NDVI_2017_data.reshape(-1,1)
NDVI_2018_flat = NDVI_2018_data.reshape(-1,1)
NDVI_2019_flat = NDVI_2019_data.reshape(-1,1)
NDVI_2020_flat = NDVI_2020_data.reshape(-1,1)
NDVI_2021_flat = NDVI_2021_data.reshape(-1,1)
Surrounding_2017_flat = Surrounding_2017_data[:-1,:-1].reshape(-1,1)
Surrounding_2018_flat = Surrounding_2018_data[:-1,:-1].reshape(-1,1)
Surrounding_2019_flat = Surrounding_2019_data[:-1,:-1].reshape(-1,1)
Surrounding_2020_flat = Surrounding_2020_data[:-1,:-1].reshape(-1,1)
Surrounding_2021_flat = Surrounding_2021_data[:-1,:-1].reshape(-1,1)

feature_vector_2017 = np.hstack((Surrounding_2017_flat, Road_density_data_flat , Road_proximity_data_flat, NDVI_2017_flat))
feature_vector_2018 = np.hstack((Surrounding_2018_flat, Road_density_data_flat , Road_proximity_data_flat, NDVI_2018_flat))
feature_vector_2019 = np.hstack((Surrounding_2019_flat, Road_density_data_flat , Road_proximity_data_flat, NDVI_2019_flat))
feature_vector_2020 = np.hstack((Surrounding_2020_flat, Road_density_data_flat , Road_proximity_data_flat, NDVI_2020_flat))
feature_vector_2021 = np.hstack((Surrounding_2021_flat, Road_density_data_flat , Road_proximity_data_flat, NDVI_2021_flat))

for yy in range(2017, 2022):
    feature_vector = globals()['feature_vector_' + str(yy)]
    for cell_index in tqdm(range(len(feature_vector)), desc='Progress: '):

      if np.isnan(feature_vector[cell_index]).any():
        survival_probs[cell_index] = np.nan
      else:
        # Predicting survival probabilities
        survival_probs[cell_index] = np.mean(RSF_model.predict_survival_function(feature_vector[cell_index].reshape(1,-1))[0].y)

    survival_prob_array = np.array(survival_probs).reshape(rows, cols)

    np.save(f'/content/drive/My Drive/COX_RSF_Paper/survival_probability_{yy}.npy', survival_prob_array)
    output_raster = f'/content/drive/My Drive/COX_RSF_Paper/survival_probability_{yy}.tif'

    with rasterio.open(
        output_raster,
        'w',
        driver='GTiff',
        height=rows,
        width=cols,
        count=1,
        dtype=survival_prob_array.dtype,
        crs=data_crs,
        transform=transform_NDVI_2021,
    ) as dst:
        dst.write(survival_prob_array, 1)

    print(f"Processing complete. The survival probability raster for the year {yy} has been saved.")

In [ ]:
BSC_df = pd.read_excel('/content/drive/My Drive/COX_RSF_Paper/BSM_2017_2021_after_considering_beehive_surrounding.xlsx')
# Transposing the DataFrame to get years as a column and classes as rows
BSC_df_transposed = BSC_df.iloc[:,:].set_index('Beekeeping suitability classes').T

# Reseting the index to use the years as a column
BSC_df_transposed.reset_index(inplace=True)
BSC_df_transposed.rename(columns={'index': 'Year'}, inplace=True)

# Manually assigning RGB colors for each class (normalize to [0, 1] range)
class_colors = {
    'Very low': (215/255, 25/255, 28/255),
    'Low': (253/255, 174/255, 97/255),
    'Moderate': (255/255, 255/255, 191/255),
    'High': (166/255, 217/255, 106/255),
    'Very high': (26/255, 150/255, 65/255),
}

plt.figure(figsize=(10, 4))
ax = plt.gca()
ax.set_facecolor("#2B3E50")  # Dark blue-gray


for class_name in BSC_df_transposed.columns[1:]:  # Skip the 'Year' column
    color = class_colors.get(class_name, (0, 0, 0))  # Default to black if class not found
    plt.plot(BSC_df_transposed['Year'], BSC_df_transposed[class_name], label=class_name, color=color, marker='o')

# Customizing the plot
plt.title('Beehive habitat suitability area percentages from 2017 to 2021', fontsize=12)
plt.xlabel('Year', fontsize=10)
plt.ylabel('Area percentage of beehive habitat suitability', fontsize=10)
plt.xticks(BSC_df_transposed['Year'], rotation=45)

plt.grid(False)
plt.legend(loc='upper right', fontsize=10)

plt.tight_layout()
plt.show()